# Rotasyonel Ticaret

Rotasyonel ticaret, en iyi performans gösteren varlıkları satın alırken düşük performans gösteren varlıkları satmayı içerir. Tahmin edebileceğiniz gibi **PyBroker** bu tür stratejilerin geriye doğru test edilmesi için mükemmel bir araçtır. O halde hadi hemen dalalım ve rotasyonel ticaret stratejimizi test etmeye başlayalım!

In [13]:
import pybroker as pyb
from pybroker import ExecContext, Strategy, StrategyConfig, YFinance

Stratejimiz, en yüksek [fiyat değişim oranı (ROC)](https://www.investopedia.com/terms/p/pricerateofchange.asp) olan hisse senetlerini sıralamayı ve satın almayı içerecektir. Başlangıç ​​olarak, [TA-Lib](https://github.com/TA-Lib/ta-lib-python) kullanarak 20 günlük bir ROC göstergesi tanımlayacağız:

In [14]:
import talib as ta

roc_20 = pyb.indicator(
    'roc_20', lambda data: ta.ROC(data.adj_close, timeperiod=20))

Şimdi stratejimizin kurallarını tanımlayalım:

- 20 günlük ROC'si en yüksek olan iki hisse senedini satın alın.
- Her hisse senedine sermayemizin %50'sini tahsis ediyoruz.
- Hisse senetlerinden herhangi biri artık ilk beş 20 günlük ROC arasında yer almıyorsa, o hisse senedini tasfiye edeceğiz.
- Bu kuralları günlük olarak değiştirin.

Yukarıdaki kurallar için konfigürasyonumuzu ve bazı parametrelerimizi ayarlayalım:

In [15]:
config = StrategyConfig(max_long_positions=2)
pyb.param('target_size', 1 / config.max_long_positions)
pyb.param('rank_threshold', 5)

5

Stratejimize devam etmek için, her hisse senedini 20 günlük ROC'sine göre en yüksekten en düşüğe doğru sıralayan bir ``sıralama`` işlevi uygulayacağız.

In [16]:
def rank(ctxs: dict[str, ExecContext]):
    scores = {
        symbol: ctx.indicator('roc_20')[-1]
        for symbol, ctx in ctxs.items()
    }
    sorted_scores = sorted(
        scores.items(), 
        key=lambda score: score[1],
        reverse=True
    )
    threshold = pyb.param('rank_threshold')
    top_scores = sorted_scores[:threshold]
    top_symbols = [score[0] for score in top_scores]
    pyb.param('top_symbols', top_symbols)

``top_symbols`` global parametresi, 20 günlük en yüksek ROC'ye sahip ilk beş hisse senedinin sembollerini içerir.

Artık hisse senetlerini ROC'lerine göre sıralamak için bir yöntemimiz olduğuna göre, rotasyonel ticareti yönetmek için bir ``rotate`` fonksiyonunu uygulamaya devam edebiliriz.

In [17]:
def rotate(ctx: ExecContext):
    if ctx.long_pos():
        if ctx.symbol not in pyb.param('top_symbols'):
            ctx.sell_all_shares()
    else:
        target_size = pyb.param('target_size')
        ctx.buy_shares = ctx.calc_target_shares(target_size)
        ctx.score = ctx.indicator('roc_20')[-1]

Şu anda tutulan hisse senedini, artık ilk beş 20 günlük ROC arasında yer almıyorsa tasfiye ederiz. Aksi takdirde, tüm hisse senetlerini 20 günlük ROC'lerine göre sıralarız ve ilk iki sıraya kadar satın alırız. Satın alma emirlerini verirken sıralama hakkında daha fazla bilgi için [Sıralama ve Konum Boyutlandırma not defterine](https://www.pybroker.com/en/latest/notebooks/4.%20Ranking%20and%20Position%20Sizing.html) bakın.

Sıralamamızı ``rank`` ile yürütmek için [set_before_exec](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy.set_before_exec) yöntemini kullanacağız. ``döndürme`` fonksiyonunu çalıştırıyorum. Bu geriye dönük test için 10 hisse senedinden oluşan bir evren kullanacağız:

In [18]:
strategy = Strategy(
    YFinance(), 
    start_date='1/1/2018', 
    end_date='1/1/2023', 
    config=config
)
strategy.set_before_exec(rank)
strategy.add_execution(rotate, [
    'FROTO.IS', 
    'DOAS.IS', 
    'FROTO.IS', 
    'EREGL.IS', 
    'ISMEN.IS',
], indicators=roc_20)
result = strategy.backtest(warmup=20)

Backtesting: 2018-01-01 00:00:00 to 2023-01-01 00:00:00

Loading bar data...
[*********************100%***********************]  4 of 4 completed
Loaded bar data: 0:00:01 

Computing indicators...


  0% (0 of 4) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--
 25% (1 of 4) |######                    | Elapsed Time: 0:00:00 ETA:   0:00:02
100% (4 of 4) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00



Test split: 2018-01-01 00:00:00 to 2022-12-30 00:00:00


  0% (0 of 1274) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--
  7% (101 of 1274) |#                    | Elapsed Time: 0:00:00 ETA:  00:00:00
 16% (211 of 1274) |###                  | Elapsed Time: 0:00:00 ETA:   0:00:00
 24% (311 of 1274) |#####                | Elapsed Time: 0:00:00 ETA:   0:00:00
 32% (411 of 1274) |######               | Elapsed Time: 0:00:00 ETA:   0:00:00
 40% (521 of 1274) |########             | Elapsed Time: 0:00:00 ETA:   0:00:00
 49% (631 of 1274) |##########           | Elapsed Time: 0:00:00 ETA:   0:00:00
 57% (731 of 1274) |############         | Elapsed Time: 0:00:00 ETA:   0:00:00
 66% (841 of 1274) |#############        | Elapsed Time: 0:00:00 ETA:   0:00:00
 75% (961 of 1274) |###############      | Elapsed Time: 0:00:00 ETA:   0:00:00
 84% (1071 of 1274) |################    | Elapsed Time: 0:00:00 ETA:   0:00:00
 92% (1181 of 1274) |##################  | Elapsed Time: 0:00:00 ETA:   0:00:00
100% (1274 of 1274) |###################


Finished backtest: 0:00:05


In [19]:
result.orders

,type,symbol,date,shares,limit_price,fill_price,fees
id,,,,,,,
1,buy,FROTO.IS,2018-01-30,794,NaN,62.48,0.0
2,buy,DOAS.IS,2018-01-30,5886,NaN,8.56,0.0
